###First aquiring the most common words

Link: https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2016/ar/ar_50k.txt


In [ ]:
file1 = open('ar_50k.txt', 'r')
Lines = file1.readlines()

In [ ]:
with open('arabic_nouns.txt', 'w') as f:
  for line in Lines:
    #print ()
    word = line.split(" ")[0]
    f.write(f"{word}\n")
  #break

Trying to Get rid of Stop Words and Stemming

Link: https://arabicstemmer.com/

In [ ]:
!pip install snowballstemmer-1.1.0.tar.gz

In [ ]:
from snowballstemmer import stemmer

In [ ]:
file1 = open('arabic_nouns.txt', 'r')
Lines = file1.readlines()

In [ ]:
stemmed_list = []
for line in Lines:
  stemmed_list.append(ar_stemmer.stemWord(line))

In [ ]:
final_nouns =list(set(stemmed_list))

In [ ]:
with open('final_nouns.txt', 'w') as f:
    for word in final_nouns:
        f.write(f"{word}\n")

###Second: Trying diffrent approaches for the model

link:https://github.com/bakrianoo/aravec

In [ ]:
import numpy as np
import gensim 
import re

In [ ]:
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

In [ ]:
#!wget "https://bakrianoo.ewr1.vultrobjects.com/aravec/full_uni_cbow_300_twitter.zip"
!wget "https://bakrianoo.ewr1.vultrobjects.com/aravec/full_uni_cbow_100_wiki.zip"

In [ ]:
#!unzip "full_uni_cbow_300_twitter.zip"
!unzip "full_uni_cbow_100_wiki.zip"

In [ ]:
#t_model = gensim.models.Word2Vec.load('full_uni_cbow_300_twitter.mdl')
t_model = gensim.models.Word2Vec.load('full_uni_cbow_100_wiki.mdl')

In [ ]:
nouns = list(t_model.wv.index_to_key)
print(nouns[:100])

In [ ]:
t_model.wv.sort_by_descending_frequency()

In [ ]:
# print(t_model.wv.vocab.keys())
nouns = list(t_model.wv.index_to_key)
print(nouns[:100])

In [ ]:
# python 3.X
token = clean_str(u'قطة')
# python 2.7
# token = clean_str('تونس'.decode('utf8', errors='ignore'))

most_similar = t_model.wv.most_similar( token, topn=50 )
for term, score in most_similar:
    print(term, score)

In [ ]:
from time import sleep

In [ ]:
def start_game(key_word = None, cold_threshold = 0.4, hot_threshold = 0.65, get_similarity = False):
    
    if key_word is None:
        # key_word = np.random.choice(nouns)
        token = clean_str(u'القطه')
    
    lag = 1.0
    print("Welcome to the game!")
    sleep(lag)
    print("I thought of a word. Your goal is to guess it!")
    sleep(lag)
    print("You will have an unlimited number of attempts to guess it")
    sleep(lag)
    print("I will say how close (hot) or far (cold) your assumption is relative to my word")
    sleep(lag)
    print("If you want to give up, enter 0 or 'give up'")
    sleep(lag)
    print("Good luck!")
    sleep(lag)
    
    win_flag = False
    give_up_flag = False
    attempts = 0
    
    while not (win_flag or give_up_flag):
        print()
        print("Type the word")
        word = input()
        
        if word == key_word:
            attempts += 1
            win_flag = True
            break
        if word == "give up" or word == '0':
            give_up_flag = True
            break
            
        if word in nouns:
            
            attempts += 1
            # similarity
            score = t_model.wv.similarity(key_word, word)
        
            if score < cold_threshold:
                print("It's cold! Your assumption is very far from the intended word")
            elif score > hot_threshold:
                print("HOT!!! Your assumption is very close to the intended word")
            else:
                print("It's warm. There is a slight connection between your assumption and the intended word")
            
            if get_similarity:
                print("similarity is " + str(np.round(score, 4)))
            
        else:
            
            print("Your word is not a singular noun!")
            print("Try again!")
    
    print()
    if win_flag:
        print("Congrats! You have won!")
        
    if give_up_flag:
        print("It was a difficult word!")
        
    print("The intended word is " + key_word)
    print("Number of attempts: " + str(attempts))

Link: https://huggingface.co/asafaya/bert-base-arabic and https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-mix

In [ ]:
!pip install transformers

In [ ]:
import numpy as np

import torch
import torch.nn.functional as f

from transformers import BertModel, BertTokenizer

from time import sleep

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
from abc import ABC
class Analyzer(ABC):
  def __init__(
      self,
      similarity_func, 
      bert_version : str,
      device, 
      ):
      self.similarity_func = similarity_func

      self.bert_version = bert_version
      self.tokenizer = BertTokenizer.from_pretrained(bert_version)
      self.model = BertModel.from_pretrained(bert_version)

      self.device=device

      self.model = self.model.eval()

  def get_word_emb(
      self, 
      token : str
      ):
      encoding = self.tokenizer(
          token, 
          padding=True, 
          return_tensors='pt'
          ).to(self.device)

      for tokens in encoding['input_ids']:
        self.tokenizer.convert_ids_to_tokens(tokens)

      with torch.no_grad():
        embed = self.model(**encoding)[0]

      avg_embed = embed.mean(dim=1)

      return avg_embed
    
  def get_similarity(self, embed1, embed2):
      return self.similarity_func(embed1, embed2)[0]

In [ ]:
with open("arabic_nouns.txt", 'r', encoding='utf-8') as f:
    nouns = f.read().split('\n')

In [ ]:
def start_game(
    key_word = None, 
    cold_threshold = 0.5, 
    hot_threshold = 0.8, 
    get_similarity = False, 
    analyzer=None
    ):
    if analyzer is None:
      embed_calc = Analyzer(
        similarity_func=torch.nn.CosineSimilarity(),
        bert_version='CAMeL-Lab/bert-base-arabic-camelbert-mix',
        device=device
      )
    else: 
      embed_calc = analyzer

    if key_word is None:
        key_word = 'كلب'#np.random.choice(nouns)
    
    key_word_emb=embed_calc.get_word_emb(token=key_word)

    lag = 1.0
    print("Welcome to the game!")
    sleep(lag)
    print("I thought of a word. Your goal is to guess it!")
    sleep(lag)
    print("You will have an unlimited number of attempts to guess it")
    sleep(lag)
    print("I will say how close (hot) or far (cold) your assumption is relative to my word")
    sleep(lag)
    print("If you want to give up, enter 0 or 'give up'")
    sleep(lag)
    print("Good luck!")
    sleep(lag)
    
    win_flag = False
    give_up_flag = False
    attempts = 0
    
    while not (win_flag or give_up_flag):
        print()
        print("Type the word")
        word = input()
        
        if word == key_word:
            attempts += 1
            win_flag = True
            break
        if word == "give up" or word == '0':
            give_up_flag = True
            break
            
        if word in nouns:
            
            attempts += 1

            word_emb=embed_calc.get_word_emb(token=word)
            score = embed_calc.get_similarity(embed1=key_word_emb, embed2=word_emb)
        
            if score < cold_threshold:
                print("It's cold! Your assumption is very far from the intended word")
            elif score > hot_threshold:
                print("HOT!!! Your assumption is very close to the intended word")
            else:
                print("It's warm. There is a slight connection between your assumption and the intended word")
            
            if get_similarity:
                print(f"similarity is {float(score.numpy()):.4f}")
            
        else:
            
            print("Your word is not a singular noun!")
            print("Try again!")
    
    print()
    if win_flag:
        print("Congrats! You have won!")
        
    if give_up_flag:
        print("It was a difficult word!")
        
    print(f"The intended word is {key_word}")
    print(f"Number of attempts: {str(attempts)}")

In [ ]:
embed_calc = Analyzer(
    similarity_func=torch.nn.CosineSimilarity(),
    bert_version='asafaya/bert-base-arabic',
    device=device
  )

In [ ]:
start_game(
    key_word = 'دمشق', 
    cold_threshold = 0.5, 
    hot_threshold = 0.88, 
    get_similarity = True, 
    analyzer=embed_calc
    )

###Another try to get the most common words

In [ ]:
file1 = open('ar_50k.txt', 'r')
Lines = file1.readlines()

In [ ]:
with open('arabic_nouns.txt', 'w') as f:
  for line in Lines:
    #print ()
    word = line.split(" ")[0]
    f.write(f"{word}\n")
  #break

In [ ]:
!wget "https://bakrianoo.ewr1.vultrobjects.com/aravec/full_uni_cbow_100_wiki.zip"

In [ ]:
!unzip "full_uni_cbow_100_wiki.zip"

In [ ]:
t_model = gensim.models.Word2Vec.load('full_uni_cbow_100_wiki.mdl')

In [ ]:
f = open("arabic_nouns.txt")
lines = f.readlines()
f.close()
nouns = [line[:-1] for line in lines]
N = len(nouns)
print (N)

In [ ]:
model_nouns = list(t_model.wv.index_to_key)

In [ ]:
model_nouns = list(t_model.wv.index_to_key)

In [ ]:
final_nouns =list(set(model_nouns).intersection(nouns))

In [ ]:
print (len(final_nouns))

In [ ]:
with open('final_nouns.txt', 'w') as f:
  for noun in final_nouns:
    #print ()
    f.write(f"{noun}\n")

###Finalizing
link : https://github.com/linuxscout/arramooz

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# read text file into pandas DataFrame
df = pd.read_csv("nouns.dict.txt", sep="\t",on_bad_lines='skip')

In [ ]:
df.head()

In [ ]:
df['مفرد/تكسير.1'].head()

In [ ]:
np.savetxt(r'nouns.txt', df['مفرد/تكسير.1'].values, fmt='%s',encoding="utf-8")